# Radar Water Observations

First step is just configuration to access radar and wofs data.

In [2]:
%%bash
cat > radar.datacube << EOF
[default]
db_hostname: agdcdev-db.nci.org.au
db_port: 6432
db_database: radar
EOF

In [ ]:
import datacube
import pytz
import functools
import digitalearthau.utils
import numpy as np

def localdate(ds, tz=pytz.timezone('Australia/Sydney')):
    """Extract date string, for local timezone, from dataset record"""
    return str(ds.time.begin.replace(tzinfo=pytz.timezone('UTC')).astimezone(tz))[:10]

We will build up a container for managing an observation.

In [25]:
class comparison:
    dc = {'wofs': datacube.Datacube(),
          'sar': datacube.Datacube(config='radar.datacube')}
    @classmethod
    def list_all(cls):
        """Returns list of all potential comparison instances"""
        return [cls(i) for i in cls.dc['sar'].find_datasets(product='s1_gamma0_scene')]
    def __init__(self, dataset_record):
        self.ds = dataset_record
        self.date = localdate(dataset_record)
    def __repr__(self):
        return '<' + self.date + '>'

x = comparison.list_all()
print(len(x))
x[:5]

2433


[<2016-12-15>, <2016-12-10>, <2016-12-22>, <2016-12-03>, <2017-04-03>]

Note that no grouping/fusing of radar images is occuring. TODO: This should be changed to ensure no pixels are double-counted, and will then need to switch to a grid workflow (rather than trying to fit an entire contiguous satellite pass into memory). As an initial prototype, it is simpler to just use the relatively few radar scenes as the unit of processing.

For now, just want to check for potential overlapping wofs scenes.

In [39]:
@functools.lru_cache(maxsize=None)
def f(self):
    """Search index for WOFS observations that potentially overlap the SAR observations"""
    return self.dc['wofs'].find_datasets(product="wofs_albers", geopolygon=self.ds.extent, time=self.date)
comparison.wofs_datasets = f

In [45]:
x2 = [i for i in x if i.wofs_datasets()]
len(x2)

802

It looks as though only a third of radar observations can potentially be paired with landsat observations, under the same-day constraint. Note there are more sophisticated constraints that could be considered instead, such as longer time-windows or even temporal brackets. However, same-day is fast to prototype with.

Next need to actually load the data, to determine whether datasets have overlapping valid pixels.

In [53]:
def f(self):
    """Retrieve full raster data"""
    radar = self.dc['sar'].load(product='s1_gamma0_scene', datasets=[self.ds], 
                                output_crs='epsg:3577', resolution=(25,-25))
    assert len(radar.time) == 1
    wofs = self.dc['wofs'].load(product='wofs_albers', datasets=self.wofs_datasets(), like=radar,
                                group_by='solar_day', fuse_func=digitalearthau.utils.wofs_fuser)
    assert len(wofs.time) == 1    
    
    return radar.isel(time=0), wofs.isel(time=0)
comparison.load = f

def f(self):
    """Extract and categorise comparable pixels"""
    radar, wofs = self.load()

    wet = wofs.water == 128
    dry = wofs.water == 0
    clear = wet | dry

    validradar = (radar.vv > 0) & (radar.vh > 0)

    overlap = clear & validradar
    self.overlap = overlap.data.sum()

    if self.overlap:
        def extract(subset):
            vv = radar.vv.values.ravel()[subset.values.ravel()]
            vh = radar.vh.values.ravel()[subset.values.ravel()]
            return vv, vh
        self.wet_vv, self.wet_vh = extract(wet & overlap)
        self.dry_vv, self.dry_vh = extract(dry & overlap)
    return self.overlap
comparison.overlay = f

In [54]:
x2[0].overlay()

6379563

In [63]:
x3 = [i for i in x2 if i.overlay()]

AssertionError: 

In [ ]:
len(x3)

At this point all of the relevant data is available. It is useful now to aggregate before analysis.

Note, it would also have been an option to coarsely bin the measurement data at an earlier stage, to limit memory consumption.

A key point is that the relationship between polarisation bands is preserved. (This would require two-dimensional binning.)

In [ ]:
wet_vv = np.concatenate([i.wet_vv for i in x3])
wet_vh = np.concatenate([i.wet_vh for i in x3])
dry_vv = np.concatenate([i.dry_vv for i in x3])
dry_vh = np.concatenate([i.dry_vh for i in x3])

Analysis will begin with visualising how the radar signal distributions depend on the wofs category.